In [1]:
import numpy as np
import pandas as pd
import re
import ssl
import nltk
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/elmedin/snap/jupyter/6/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /home/elmedin/snap/jupyter/6/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
data = pd.read_csv('./netflix_titles.csv')
data.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,TV Show,3%,NaN,"João Miguel, Bianca Comparato, Michel Gomes, R...",Brazil,"August 14, 2020",2020,TV-MA,4 Seasons,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...
1,s2,Movie,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",Mexico,"December 23, 2016",2016,TV-MA,93 min,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...
2,s3,Movie,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...",Singapore,"December 20, 2018",2011,R,78 min,"Horror Movies, International Movies","When an army recruit is found dead, his fellow..."
3,s4,Movie,9,Shane Acker,"Elijah Wood, John C. Reilly, Jennifer Connelly...",United States,"November 16, 2017",2009,PG-13,80 min,"Action & Adventure, Independent Movies, Sci-Fi...","In a postapocalyptic world, rag-doll robots hi..."
4,s5,Movie,21,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",United States,"January 1, 2020",2008,PG-13,123 min,Dramas,A brilliant group of students become card-coun...


In [3]:
data.groupby('type').count()

,show_id,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
type,,,,,,,,,,,
Movie,5377,5377,5214,4951,5147,5377,5377,5372,5377,5377,5377
TV Show,2410,2410,184,2118,2133,2400,2410,2408,2410,2410,2410


In [4]:
data = data.dropna(subset=['cast', 'country', 'rating'])

In [5]:
movies = data[data['type'] == 'Movie'].reset_index()
movies = movies.drop(['index', 'show_id', 'type', 'date_added', 'release_year', 'duration', 'description'], axis=1)
movies.head()

,title,director,cast,country,rating,listed_in
0,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",Mexico,TV-MA,"Dramas, International Movies"
1,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...",Singapore,R,"Horror Movies, International Movies"
2,9,Shane Acker,"Elijah Wood, John C. Reilly, Jennifer Connelly...",United States,PG-13,"Action & Adventure, Independent Movies, Sci-Fi..."
3,21,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",United States,PG-13,Dramas
4,122,Yasir Al Yasiri,"Amina Khalil, Ahmed Dawood, Tarek Lotfy, Ahmed...",Egypt,TV-MA,"Horror Movies, International Movies"


In [6]:
tv = data[data['type'] == 'TV Show'].reset_index()
tv = tv.drop(['index', 'show_id', 'type', 'date_added', 'release_year', 'duration', 'description'], axis=1)
tv.head()

,title,director,cast,country,rating,listed_in
0,3%,NaN,"João Miguel, Bianca Comparato, Michel Gomes, R...",Brazil,TV-MA,"International TV Shows, TV Dramas, TV Sci-Fi &..."
1,46,Serdar Akar,"Erdal Beşikçioğlu, Yasemin Allen, Melis Birkan...",Turkey,TV-MA,"International TV Shows, TV Dramas, TV Mysteries"
2,1983,NaN,"Robert Więckiewicz, Maciej Musiał, Michalina O...","Poland, United States",TV-MA,"Crime TV Shows, International TV Shows, TV Dramas"
3,​SAINT SEIYA: Knights of the Zodiac,NaN,"Bryson Baugus, Emily Neves, Blake Shepard, Pat...",Japan,TV-14,"Anime Series, International TV Shows"
4,#blackAF,NaN,"Kenya Barris, Rashida Jones, Iman Benson, Genn...",United States,TV-MA,TV Comedies


In [7]:
actors = []

for i in movies['cast']:
    actor = re.split(r', \s*', i)
    actors.append(actor)
    
flat_list = []
for sublist in actors:
    for item in sublist:
        flat_list.append(item)
        
actors_list = sorted(set(flat_list))

binary_actors = [[0] * 0 for i in range(len(set(flat_list)))]

for i in movies['cast']:
    k = 0
    for j in actors_list:
        if j in i:
            binary_actors[k].append(1.0)
        else:
            binary_actors[k].append(0.0)
        k+=1
        
binary_actors = pd.DataFrame(binary_actors).transpose()
        
directors = []

for i in movies['director']:
    if pd.notna(i):
        director = re.split(r', \s*', i)
        directors.append(director)
    
flat_list2 = []
for sublist in directors:
    for item in sublist:
        flat_list2.append(item)
        
directors_list = sorted(set(flat_list2))

binary_directors = [[0] * 0 for i in range(len(set(flat_list2)))]

for i in movies['director']:
    k = 0
    for j in directors_list:
        if pd.isna(i):
            binary_directors[k].append(0.0)
        elif j in i:
            binary_directors[k].append(1.0)
        else:
            binary_directors[k].append(0.0)
        k+=1
        
binary_directors = pd.DataFrame(binary_directors).transpose()
        
countries = []

for i in movies['country']:
    country = re.split(r', \s*', i)
    countries.append(country)
    
flat_list3 = []
for sublist in countries:
    for item in sublist:
        flat_list3.append(item)
        
countries_list = sorted(set(flat_list3))

binary_countries = [[0] * 0 for i in range(len(set(flat_list3)))]

for i in movies['country']:
    k = 0
    for j in countries_list:
        if j in i:
            binary_countries[k].append(1.0)
        else:
            binary_countries[k].append(0.0)
        k+=1
        
binary_countries = pd.DataFrame(binary_countries).transpose()

genres = []

for i in movies['listed_in']:
    genre = re.split(r', \s*', i)
    genres.append(genre)
    
flat_list4 = []
for sublist in genres:
    for item in sublist:
        flat_list4.append(item)
        
genres_list = sorted(set(flat_list4))

binary_genres = [[0] * 0 for i in range(len(set(flat_list4)))]

for i in movies['listed_in']:
    k = 0
    for j in genres_list:
        if j in i:
            binary_genres[k].append(1.0)
        else:
            binary_genres[k].append(0.0)
        k+=1
        
binary_genres = pd.DataFrame(binary_genres).transpose()

ratings = []

for i in movies['rating']:
    ratings.append(i)

ratings_list = sorted(set(ratings))

binary_ratings = [[0] * 0 for i in range(len(set(ratings_list)))]

for i in movies['rating']:
    k = 0
    for j in ratings_list:
        if j in i:
            binary_ratings[k].append(1.0)
        else:
            binary_ratings[k].append(0.0)
        k+=1
        
binary_ratings = pd.DataFrame(binary_ratings).transpose()

In [8]:
binary = pd.concat([binary_actors, binary_directors, binary_countries, binary_genres], axis=1, ignore_index=True)
binary

,0,1,2,3,4,5,6,7,8,9,...,26570,26571,26572,26573,26574,26575,26576,26577,26578,26579
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4756,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4757,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4758,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4759,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
actors2 = []

for i in tv['cast']:
    actor2 = re.split(r', \s*', i)
    actors2.append(actor2)
    
    flat_list5 = []
    for sublist in actors2:
        for item in sublist: 
            flat_list5.append(item)
        
actors_list2 = sorted(set(flat_list5))

binary_actors2 = [[0] * 0 for i in range(len(set(flat_list5)))]

for i in tv['cast']:
    k = 0
    for j in actors_list2:
        if j in i:
            binary_actors2[k].append(1.0)
        else:
            binary_actors2[k].append(0.0)
        k+=1

binary_actors2 = pd.DataFrame(binary_actors2).transpose()

countries2 = []
for i in tv['country']:
    country2 = re.split(r', \s*', i)
    countries2.append(country2)
    
flat_list6 = []
for sublist in countries2:
    for item in sublist: 
        flat_list6.append(item)
        
countries_list2 = sorted(set(flat_list6))

binary_countries2 = [[0] * 0 for i in range(len(set(flat_list6)))]

for i in tv['country']:
    k = 0
    for j in countries_list2:
        if j in i:
            binary_countries2[k].append(1.0)
        else:
            binary_countries2[k].append(0.0)
        k+=1
        
binary_countries2 = pd.DataFrame(binary_countries2).transpose()

genres2 = []

for i in tv['listed_in']:
    genre2 = re.split(r', \s*', i)
    genres2.append(genre2)
    
flat_list7 = []
for sublist in genres2:
    for item in sublist:
        flat_list7.append(item)
        
genres_list2 = sorted(set(flat_list7))

binary_genres2 = [[0] * 0 for i in range(len(set(flat_list7)))]

for i in tv['listed_in']:
    k = 0
    for j in genres_list2:
        if j in i: 
            binary_genres2[k].append(1.0)
        else:
            binary_genres2[k].append(0.0)
        k+=1
        
binary_genres2 = pd.DataFrame(binary_genres2).transpose()

ratings2 = []

for i in tv['rating']:
    ratings2.append(i)
    
ratings_list2 = sorted(set(ratings2))

binary_ratings2 = [[0] * 0 for i in range(len(set(ratings_list2)))]

for i in tv['rating']:
    k = 0
    for j in ratings_list2:
        if j in i:
            binary_ratings2[k].append(1.0)
        else:
            binary_ratings2[k].append(0.0)
        k+=1
        
binary_ratings2 = pd.DataFrame(binary_ratings2).transpose()

In [10]:
binary2 = pd.concat([binary_actors2, binary_countries2, binary_genres2], axis=1, ignore_index=True)
binary2

,0,1,2,3,4,5,6,7,8,9,...,12741,12742,12743,12744,12745,12746,12747,12748,12749,12750
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1886,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1887,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
1888,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1889,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [11]:
def recommender(search):
    cs_list = []
    binary_list = []
    if search in movies['title'].values:
        idx = movies[movies['title'] == search].index.item()
        for i in binary.iloc[idx]:
            binary_list.append(i)
        point1 = np.array(binary_list).reshape(1, -1)
        point1 = [val for sublist in point1 for val in sublist]
        for j in range(len(movies)):
            binary_list2 = []
            for k in binary.iloc[j]:
                binary_list2.append(k)
            point2 = np.array(binary_list2).reshape(1, -1)
            point2 = [val for sublist in point2 for val in sublist]
            dot_product = np.dot(point1, point2)
            norm_1 = np.linalg.norm(point1)
            norm_2 = np.linalg.norm(point2)
            cos_sim = dot_product / (norm_1 * norm_2)
            cs_list.append(cos_sim)
        movies_copy = movies.copy()
        movies_copy['cos_sim'] = cs_list
        results = movies_copy.sort_values('cos_sim', ascending=False)
        results = results[results['title'] != search]
        top_results = results.head(5)
        return(top_results)
    elif search in tv['title'].values:
        idx = tv[tv['title'] == search].index.item()
        for i in binary2.iloc[idx]:
            binary_list.append(i)
        point1 = np.array(binary_list).reshape(1, -1)
        point1 = [val for sublist in point1 for val in sublist]
        for j in range(len(tv)):
            binary_list2 = []
            for k in binary2.iloc[j]:
                binary_list2.append(k)
            point2 = np.array(binary_list2).reshape(1, -1)
            point2 = [val for sublist in point2 for val in sublist]
            dot_product = np.dot(point1, point2)
            norm_1 = np.linalg.norm(point1)
            norm_2 = np.linalg.norm(point2)
            cos_sim = dot_product / (norm_1 * norm_2)
            cs_list.append(cos_sim)
        tv_copy = tv.copy()
        tv_copy['cos_sim'] = cs_list
        results = tv_copy.sort_values('cos_sim', ascending=False)
        results = results[results['title'] != search]
        top_results = results.head(5)
        return(top_results)
    else:
        return('Title can not be found in the given dataset. Please check spelling or enter a different title.')

In [12]:
recommender('The Conjuring')

,title,director,cast,country,rating,listed_in,cos_sim
1868,Insidious,James Wan,"Patrick Wilson, Rose Byrne, Lin Shaye, Ty Simp...","United States, Canada, United Kingdom",PG-13,"Horror Movies, Thrillers",0.388922
968,Creep,Patrick Brice,"Mark Duplass, Patrick Brice",United States,R,"Horror Movies, Independent Movies, Thrillers",0.377964
1844,In the Tall Grass,Vincenzo Natali,"Patrick Wilson, Laysla De Oliveira, Avery Whit...","Canada, United States",TV-MA,"Horror Movies, Thrillers",0.370625
969,Creep 2,Patrick Brice,"Mark Duplass, Desiree Akhavan, Karan Soni",United States,TV-MA,"Horror Movies, Independent Movies, Thrillers",0.356348
1077,Desolation,Sam Patton,"Jaimi Paige, Alyshia Ochse, Toby Nichols, Clau...",United States,TV-MA,"Horror Movies, Thrillers",0.356348


In [13]:
recommender('Bad Boys')

,title,director,cast,country,rating,listed_in,cos_sim
467,Bad Boys II,Michael Bay,"Will Smith, Martin Lawrence, Jordi Mollà, Gabr...",United States,R,"Action & Adventure, Comedies",0.552052
928,Coffee & Kareem,Michael Dowse,"Ed Helms, Taraji P. Henson, Terrence Little Ga...",United States,TV-MA,"Action & Adventure, Comedies",0.273861
2494,Men in Black,Barry Sonnenfeld,"Tommy Lee Jones, Will Smith, Linda Fiorentino,...",United States,PG-13,"Action & Adventure, Comedies, Sci-Fi & Fantasy",0.266667
2495,Men in Black II,Barry Sonnenfeld,"Tommy Lee Jones, Will Smith, Rip Torn, Lara Fl...",United States,PG-13,"Action & Adventure, Comedies, Sci-Fi & Fantasy",0.266667
4667,Wild Wild West,Barry Sonnenfeld,"Will Smith, Kevin Kline, Kenneth Branagh, Salm...",United States,PG-13,"Action & Adventure, Comedies, Sci-Fi & Fantasy",0.266667


In [14]:
recommender('Breaking Bad')

,title,director,cast,country,rating,listed_in,cos_sim
169,Better Call Saul,NaN,"Bob Odenkirk, Jonathan Banks, Michael McKean, ...",United States,TV-MA,"Crime TV Shows, TV Comedies, TV Dramas",0.521749
1459,The Assassination of Gianni Versace,NaN,"Edgar Ramírez, Darren Criss, Ricky Martin, Pen...",United States,TV-MA,"Crime TV Shows, TV Dramas, TV Thrillers",0.430331
1775,Unsolved,NaN,"Josh Duhamel, Jimmi Simpson, Bokeem Woodbine",United States,TV-MA,"Crime TV Shows, TV Dramas",0.390360
537,Get Shorty,NaN,"Ray Romano, Chris O'Dowd",United States,TV-MA,"Crime TV Shows, TV Comedies, TV Dramas",0.390360
1416,Surviving R. Kelly,NaN,R. Kelly,United States,TV-MA,"Crime TV Shows, Docuseries",0.346410


In [15]:
recommender('Friends')

,title,director,cast,country,rating,listed_in,cos_sim
455,Episodes,NaN,"Matt LeBlanc, Tamsin Greig, Stephen Mangan, Ka...","United Kingdom, United States",TV-MA,"Classic & Cult TV, TV Comedies",0.476731
1133,Pee-wee's Playhouse,NaN,Paul Reubens,United States,TV-PG,"Classic & Cult TV, Kids' TV, TV Comedies",0.424264
513,Frasier,NaN,"Kelsey Grammer, Jane Leeves, David Hyde Pierce...",United States,TV-PG,"Classic & Cult TV, TV Comedies",0.400000
910,Man with a Plan,NaN,"Matt LeBlanc, Liza Snyder, Grace Kaufman, Matt...",United States,TV-PG,TV Comedies,0.400000
1550,The Honeymoon Stand Up Special,NaN,"Natasha Leggero, Moshe Kasher",United States,TV-MA,"Stand-Up Comedy & Talk Shows, TV Comedies",0.387298


In [16]:
movies_des = data[data['type'] == 'Movie'].reset_index()
movies_des = movies_des[['title', 'description']]
movies_des.head()

,title,description
0,7:19,After a devastating earthquake hits Mexico Cit...
1,23:59,"When an army recruit is found dead, his fellow..."
2,9,"In a postapocalyptic world, rag-doll robots hi..."
3,21,A brilliant group of students become card-coun...
4,122,"After an awful accident, a couple admitted to ..."


In [17]:
tv_des = data[data['type'] == 'TV Show'].reset_index()
tv_des = tv_des[['title', 'description']]
tv_des.head()

,title,description
0,3%,In a future where the elite inhabit an island ...
1,46,A genetics professor experiments with a treatm...
2,1983,"In this dark alt-history thriller, a naïve law..."
3,​SAINT SEIYA: Knights of the Zodiac,Seiya and the Knights of the Zodiac rise again...
4,#blackAF,Kenya Barris and his family navigate relations...


In [18]:
filtered_movies = []
movies_words = []

for text in movies_des['description']:
    text_tokens = word_tokenize(text)
    tokens_without_sw = [word.lower() for word in text_tokens if not word in stopwords.words()]
    movies_words.append(tokens_without_sw)
    filtered = (" ").join(tokens_without_sw)
    filtered_movies.append(filtered)

movies_words = [val for sublist in movies_words for val in sublist]
movies_words = sorted(set(movies_words))
movies_des['description_filtered'] = filtered_movies
movies_des.head()

,title,description,description_filtered
0,7:19,After a devastating earthquake hits Mexico Cit...,after devastating earthquake hits mexico city ...
1,23:59,"When an army recruit is found dead, his fellow...","when army recruit found dead , fellow soldiers..."
2,9,"In a postapocalyptic world, rag-doll robots hi...","in postapocalyptic world , rag-doll robots hid..."
3,21,A brilliant group of students become card-coun...,a brilliant group students become card-countin...
4,122,"After an awful accident, a couple admitted to ...","after awful accident , couple admitted grisly ..."


In [19]:
filtered_tv = []
tv_words = []

for text in tv_des['description']:
    text_tokens = word_tokenize(text)
    tokens_without_sw = [word.lower() for word in text_tokens if not word in stopwords.words()]
    tv_words.append(tokens_without_sw)
    filtered = (' ').join(tokens_without_sw)
    filtered_tv.append(filtered)
    
tv_words = [val for sublist in tv_words for val in sublist]
tv_words = sorted(set(tv_words))
tv_des['description_filtered'] = filtered_tv
tv_des.head()

,title,description,description_filtered
0,3%,In a future where the elite inhabit an island ...,in future elite inhabit island paradise far cr...
1,46,A genetics professor experiments with a treatm...,a genetics professor experiments treatment com...
2,1983,"In this dark alt-history thriller, a naïve law...","in dark alt-history thriller , naïve law stude..."
3,​SAINT SEIYA: Knights of the Zodiac,Seiya and the Knights of the Zodiac rise again...,seiya knights zodiac rise protect reincarnatio...
4,#blackAF,Kenya Barris and his family navigate relations...,"kenya barris family navigate relationships , r..."


In [20]:
movie_word_binary = [[0] * 0 for i in range(len(set(movies_words)))]

for des in movies_des['description_filtered']:
    k = 0
    for word in movies_words:
        if word in des:
            movie_word_binary[k].append(1.0)
        else:
            movie_word_binary[k].append(0.0)
        k+=1
        
movie_word_binary = pd.DataFrame(movie_word_binary).transpose()

In [21]:
tv_word_binary = [[0] * 0 for i in range(len(set(tv_words)))]

for des in tv_des['description_filtered']:
    k = 0
    for word in tv_words:
        if word in des:
            tv_word_binary[k].append(1.0)
        else:
            tv_word_binary[k].append(0.0)
        k+=1
    
tv_word_binary = pd.DataFrame(tv_word_binary).transpose()

In [22]:
def recommender2(search):
    cs_list = []
    binary_list = []
    if search in movies_des['title'].values:
        idx = movies_des[movies_des['title'] == search].index.item()
        for i in movie_word_binary.iloc[idx]:
            binary_list.append(i)
        point1 = np.array(binary_list).reshape(1, -1)
        point1 = [val for sublist in point1 for val in sublist]    
        for j in range(len(movies_des)):
            binary_list2 = []
            for k in movie_word_binary.iloc[j]:
                binary_list2.append(k)
            point2 = np.array(binary_list2).reshape(1, -1)
            point2 = [val for sublist in point2 for val in sublist]
            dot_product = np.dot(point1, point2)
            norm_1 = np.linalg.norm(point1)
            norm_2 = np.linalg.norm(point2)
            cos_sim = dot_product / (norm_1 * norm_2)
            cs_list.append(cos_sim)
        movies_copy = movies_des.copy()
        movies_copy['cos_sim'] = cs_list
        results = movies_copy.sort_values('cos_sim', ascending=False)
        results = results[results['title'] != search]    
        top_results = results.head(5)
        return(top_results)
    elif search in tv_des['title'].values:
        idx = tv_des[tv_des['title'] == search].index.item()
        for i in tv_word_binary.iloc[idx]:
            binary_list.append(i)
        point1 = np.array(binary_list).reshape(1, -1)
        point1 = [val for sublist in point1 for val in sublist]
        for j in range(len(tv)):
            binary_list2 = []
            for k in tv_word_binary.iloc[j]:
                binary_list2.append(k)
            point2 = np.array(binary_list2).reshape(1, -1)
            point2 = [val for sublist in point2 for val in sublist]
            dot_product = np.dot(point1, point2)
            norm_1 = np.linalg.norm(point1)
            norm_2 = np.linalg.norm(point2)
            cos_sim = dot_product / (norm_1 * norm_2)
            cs_list.append(cos_sim)
        tv_copy = tv_des.copy()
        tv_copy['cos_sim'] = cs_list
        results = tv_copy.sort_values('cos_sim', ascending=False)
        results = results[results['title'] != search]    
        top_results = results.head(5)
        return(top_results)
    else:
        return('Title can not be found in the given dataset. Please check spelling or enter a different title.')

In [23]:
pd.options.display.max_colwidth = 300
recommender2('The Conjuring')

,title,description,description_filtered,cos_sim
1632,Hard Lessons,"This drama based on real-life events tells the story of George McKenna, the tough, determined new principal of a notorious Los Angeles high school.","this drama based real-life events tells story george mckenna , tough , determined new principal notorious los angeles high school .",0.489419
3335,Sat Sri Akal,"Based on true events, this moving story centers on a Punjabi family whose celebration of their faith endures in the face of conflicting attitudes.","based true events , moving story centers punjabi family whose celebration faith endures conflicting attitudes .",0.478650
2549,Mirai,"Unhappy after his new baby sister displaces him, four-year-old Kun begins meeting people and pets from his family's history in their unique house.","unhappy new baby sister displaces , four-year-old kun begins meeting people pets family 's history unique house .",0.478345
3910,The Eyes of My Mother,"At the remote farmhouse where she once witnessed a traumatic childhood event, a young woman develops a grisly fascination with violence.","at remote farmhouse witnessed traumatic childhood event , young woman develops grisly fascination violence .",0.470605
3578,Standoff,"After witnessing an assassin's slaughter, a young girl holes up in a farmhouse with a suicidal vet, who must use wits and guts to fend off the killer.","after witnessing assassin 's slaughter , young girl holes farmhouse suicidal vet , must use wits guts fend killer .",0.460628


In [24]:
recommender2('Bad Boys')


,title,description,description_filtered,cos_sim
4105,The Midnight Sky,"In the aftermath of a global catastrophe, a lone scientist in the Arctic races to contact a crew of astronauts with a warning not to return to Earth.","in aftermath global catastrophe , lone scientist arctic races contact crew astronauts warning return earth .",0.422178
467,Bad Boys II,"In this hyperkinetic sequel, a pair of Miami narcotics cops investigate a Cuban drug kingpin's connection to Florida's influx of Ecstasy.","in hyperkinetic sequel , pair miami narcotics cops investigate cuban kingpin 's connection florida 's influx ecstasy .",0.421117
1673,Hellboy,"This fast-paced action flick tells the dark story of Hellboy, a charismatic demon who's put on Earth by the Nazis to terrorize the rest of the world.","this fast-paced action flick tells dark story hellboy , charismatic demon 's put earth nazis terrorize rest world .",0.410632
894,Chup Chup Ke,"Mistaking a broke, small-time hustler for a millionaire, two bumbling fishermen hatch a scheme to use him to help pay off their debts.","mistaking broke , small-time hustler millionaire , two bumbling fishermen hatch scheme use help pay debts .",0.400892
3396,Senna,"This fast-paced documentary profiles Ayrton Senna, one of the greatest drivers in the history of Formula One racing, and a hero in his native Brazil.","this fast-paced documentary profiles ayrton senna , greatest drivers history formula one racing , hero native brazil .",0.394771


In [25]:
recommender2('Breaking Bad')

,title,description,description_filtered,cos_sim
996,Mr. Iglesias,Hilarious high school teacher Gabriel Iglesias tries to make a difference in the lives of some smart but underperforming students at his alma mater.,hilarious high school teacher gabriel iglesias tries make difference lives smart underperforming students alma mater .,0.467099
931,Marvel's The Punisher,A former Marine out to punish the criminals responsible for his family's murder finds himself ensnared in a military conspiracy.,a former marine punish criminals responsible family 's murder finds ensnared military conspiracy .,0.456435
1297,Servant of the People,"After a Ukrainian high school teacher's tirade against government corruption goes viral on social media, he finds himself the country's new president.","after ukrainian high school teacher 's tirade government corruption goes viral social media , finds country 's new president .",0.453632
443,El Reemplazante,A former high-ranking financial executive finds redemption and romance when he's paroled after a prison sentence and becomes a math teacher.,a former high-ranking financial executive finds redemption romance 's paroled prison sentence becomes math teacher .,0.443622
132,Bad Education,"A history teacher at the posh Abbey Grove, Alfie Wickers is something truly special: He's his school's, if not England's, single worst educator.","a history teacher posh abbey grove , alfie wickers something truly special : he 's school 's , england 's , single worst educator .",0.412479


In [26]:
recommender2('Friends')

,title,description,description_filtered,cos_sim
1700,Thomas and Friends,"This animated children's series follows the adventures of Thomas, a cheerful blue tank engine who lives on the island of Sodor.","this animated children 's series follows adventures thomas , cheerful blue tank engine lives island sodor .",0.419292
211,Bondi Rescue,"This reality series follows elite professional lifeguards on Sydney's Bondi Beach, as they take on everything from animal stings to criminals.","this reality series follows elite professional lifeguards sydney 's bondi beach , everything animal stings criminals .",0.403376
593,H,"At a dysfunctional hospital in Paris, three bumbling, eccentric medical employees embark on zany misadventures with surgical imprecision.","at dysfunctional hospital paris , three bumbling , eccentric medical employees embark zany misadventures surgical imprecision .",0.403101
1266,Saint Seiya: The Lost Canvas,This anime adventure follows the battle between a saint of Athena and an avatar of Hades who's working on a painting that could destroy the world.,this anime adventure follows battle saint athena avatar hades 's working painting could destroy world .,0.401772
131,Bad Blood,"This sprawling crime drama follows the true story of the Rizzuto family and its associates, who presided over organized crime in Montreal for decades.","this sprawling crime drama follows true story rizzuto family associates , presided organized crime montreal decades .",0.398790
